 TRY
top_k_categorical_accuracy
https://stackoverflow.com/questions/47887533/keras-convolution-along-samples
https://keras.io/layers/wrappers/#timedistributed

# Fine tuning one look

In [1]:
input_width = 160
input_height = 100
channels = 2
class_number = 12
data_path = "D:\\Python\\Wormax_learn2\\preprocessed_data_local_notshuffled\\"

In [2]:
from keras.models import Sequential, Model
from keras import layers, models
import keras.backend as K
from keras.applications import Xception

conv_base = Xception(weights='imagenet',
                    include_top=False,
                    input_shape=(input_height, input_width, 3))

def actual_acc(y_true, y_pred):
    return K.equal(K.argmax(y_pred), K.argmax(y_true))


def define_model():
    model = models.Sequential()
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    
    model.add(layers.Dense(class_number, activation='softmax'))
   
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
                  metrics=[actual_acc])
    
    model.summary()
    return model

Q:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ImportError: Could not find 'cudart64_90.dll'. TensorFlow requires that this DLL be installed in a directory that is named in your %PATH% environment variable. Download and install CUDA 9.0 from this URL: https://developer.nvidia.com/cuda-toolkit

In [ ]:
#conv_base.summary()
conv_base.trainable = False
model = define_model()

In [ ]:
# little prepocessing
from math import atan2, pi

def get_angle(x, y):
    return atan2(y, x)

def get_direction(x, y, n_classes = 12):
    return round(get_angle(x, y)/2/pi*n_classes)%n_classes

In [ ]:
import numpy as np
import os
import random
from functools import reduce
from keras.utils import to_categorical

# Training and validation
data_ratio = 0.7

def generator(data_dir, num_classes, role, shuffle=True, batch_size=128):
    
    listdir = []
    listdir = filter(lambda x: os.path.isfile, os.listdir(data_dir))
    listdir = np.array(list(listdir))
    random.shuffle(listdir)
    
    #print('Found {} files for {}'.format(len(listdir), role))
    
    file_i = 0
    while 1:
        data = np.load(data_dir + listdir[file_i])
        file_i = (file_i+1) if file_i+1<len(listdir) else 0
        
        if role == 'train':
            data = data[:int(round(len(data)*data_ratio))]
        elif role == 'validation':
            data = data[int(round(len(data)*data_ratio)):]
        else:
            raise 'bad role parameter'
        
        if shuffle:
            np.random.shuffle(data)
        
        for i in range(0, len(data)-1-batch_size, batch_size):
            samples = data[i:i+batch_size, 0]
            targets = []
            for j in range(batch_size):
                targets.append(
                    to_categorical(get_direction(*data[i+1+j][1][:2]), num_classes=num_classes)
                )
            
            #print(np.array(targets).shape)
            
            # Weird reshape cuz bug
            samples2 = np.zeros((batch_size, input_height, input_width, 3))
            for j, sample in enumerate(samples):
                samples2[j] = sample
            
            yield samples2, np.array(targets)

In [ ]:
import numpy as np

train_generator = generator(data_path, class_number, 'train', batch_size=128)
validation_generator = generator(data_path, class_number, 'validation', batch_size=128)

print(next(train_generator)[0].shape)

In [ ]:
model.evaluate_generator(validation_generator, 100)

In [ ]:
steps_per_epoch = 200
history = model.fit_generator(train_generator,
                            steps_per_epoch=steps_per_epoch,
                            epochs=100,
                            validation_data=validation_generator,
                            validation_steps=int(round(steps_per_epoch/data_ratio*(1-data_ratio))),
                            shuffle=True
                             )

model.save('worm1_transfer_single_delme.h5')


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['actual_acc']
val_acc = history.history['val_actual_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

def smooth_curve(points, factor=0.8):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

plt.plot(epochs,
smooth_curve(acc), 'bo', label='Smoothed training acc')
plt.plot(epochs,
smooth_curve(val_acc), 'b', label='Smoothed validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs,
smooth_curve(loss), 'bo', label='Smoothed training loss')
plt.plot(epochs,
smooth_curve(val_loss), 'b', label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()